# Slight Detour: What is Semantic Similarity?

Semantic similarity is a concept in natural language processing and computational linguistics that measures how similar two pieces of text are in meaning, rather than just looking at lexical (word-level) matches.

At its core, semantic similarity aims to capture when two different texts express similar ideas, concepts, or information, even if they use completely different words. For example, "The automobile won't start" and "My car isn't working" have high semantic similarity despite using different vocabulary.

![Semantic SImilarity](./images/semantic_similarity.png "Semantic Similarity")

In [ ]:
! pip install faiss-cpu

In [30]:
import requests
import numpy as np
import faiss

In [31]:
texts = [
    "The cat sat on the mat",
    "A feline was resting on a rug",
    "Dogs are great pets",
    "I love having a canine companion",
    "Paris is the capital of France",
    "The Eiffel Tower is in Paris"
]

In [32]:
# Using Nomic model served locally via Ollama for embedding
# Ollama is a friend --> https://ollama.com/
def get_embeddings_from_ollama(text, model="nomic-embed-text"):
    url = "http://localhost:11434/api/embeddings"
    
    payload = {
        "model": model,
        "prompt": text
    }
    
    response = requests.post(url, json=payload)
    return np.array(response.json()["embedding"], dtype=np.float32)


In [14]:
# Generate embeddings for all texts
embeddings = []
for text in texts:
    embedding = get_embeddings_from_ollama(text)
    embeddings.append(embedding)

In [15]:
# Convert list to numpy array and create FAISS index
embeddings = np.array(embeddings)

dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

In [34]:
# Create an embedding for input 
# We will compare this sentence to the ones above
query_text = "A cat is sitting on a carpet"
query_vector = get_embeddings_from_ollama(query_text)
query_vector = query_vector.reshape(1, -1)  # Reshape for FAISS

In [ ]:
# Print out for fun
print(f"Vector shape: {query_vector.shape}")
print(f"Vector type: {query_vector.dtype}")
print(query_vector)

In [ ]:
# Search for similarity with ALL texts in the index
k = len(texts)  # Return all results
distances, indices = index.search(query_vector, k)

# Display results
print(f"Query: {query_text}\n")
print("All texts ranked by similarity:")
for i, (dist, idx) in enumerate(zip(distances[0], indices[0])):
    print(f"{i+1}. \"{texts[idx]}\" (Distance: {dist:.4f})")